In [1]:
import sys
import os

# Add the parent directory to sys.path if models.py is in the parent folder
sys.path.append(os.path.abspath('..'))

from models import GatheringModel, read_input, compute_multiphase_pressure_drop
import pandas as pd
from gamspy import Options, Ord

In [2]:
m = GatheringModel("Prototipo", data= read_input("toy_problem.xlsx"))

--- Job _d0nFgZGBQ4eyFQwyZhTk0A.gms Start 09/15/25 17:15:19 50.4.1 84b10359 WEX-WEI x86 64bit/MS Windows
--- Applying:
    C:\Users\Diego\Desktop\ExxonMobil\LM_Gathering\venv\Lib\site-packages\gamspy_base\gmsprmNT.txt
    C:\Users\Diego\Documents\GAMS\gamsconfig.yaml
--- GAMS Parameters defined
    Input C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpt13ke5f6\_d0nFgZGBQ4eyFQwyZhTk0A.gms
    Output C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpt13ke5f6\_d0nFgZGBQ4eyFQwyZhTk0A.lst
    ScrDir C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpt13ke5f6\tmpt07pda9n\
    SysDir C:\Users\Diego\Desktop\ExxonMobil\LM_Gathering\venv\Lib\site-packages\gamspy_base\
    LogOption 3
    Trace C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpt13ke5f6\_d0nFgZGBQ4eyFQwyZhTk0A.txt
    License C:\Users\Diego\Documents\GAMSPy\gamspy_license.txt
    OptFile 0
    OptDir C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpt13ke5f6\
    LimRow 0
    LimCol 0
    TraceOpt 3
    SolPrint 0


In [3]:
m.solution_algorithm(scenario_name="prueba4", max_iterations=3, solver_opts={"solver": "gurobi", "gap": 0.0001, "max_time": 100.0})

--- Iteration 1 ---
--- _d0nFgZGBQ4eyFQwyZhTk0A.gms(1039) 11 Mb
--- Job _d0nFgZGBQ4eyFQwyZhTk0A.gms Start 09/15/25 17:15:49 50.4.1 84b10359 WEX-WEI x86 64bit/MS Windows
--- Applying:
    C:\Users\Diego\Desktop\ExxonMobil\LM_Gathering\venv\Lib\site-packages\gamspy_base\gmsprmNT.txt
    C:\Users\Diego\Documents\GAMS\gamsconfig.yaml
--- GAMS Parameters defined
    MIQCP gurobi
    Input C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpt13ke5f6\_d0nFgZGBQ4eyFQwyZhTk0A.gms
    Output C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpt13ke5f6\_d0nFgZGBQ4eyFQwyZhTk0A.lst
    ScrDir C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpt13ke5f6\tmpt07pda9n\
    SysDir C:\Users\Diego\Desktop\ExxonMobil\LM_Gathering\venv\Lib\site-packages\gamspy_base\
    LogOption 3
    Trace C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpt13ke5f6\_d0nFgZGBQ4eyFQwyZhTk0A.txt
    License C:\Users\Diego\Documents\GAMSPy\gamspy_license.txt
    OptFile 0
    OptDir C:\Users\Public\Documents\Wondershare\

[MODEL - WARNING] The solve was interrupted! Solve status: ResourceInterrupt. For further information, see https://gamspy.readthedocs.io/en/latest/reference/gamspy._model.html#gamspy.SolveStatus.


--- _d0nFgZGBQ4eyFQwyZhTk0A.gms(1109) 159 Mb
--- Job _d0nFgZGBQ4eyFQwyZhTk0A.gms Start 09/15/25 17:17:39 50.4.1 84b10359 WEX-WEI x86 64bit/MS Windows
--- Applying:
    C:\Users\Diego\Desktop\ExxonMobil\LM_Gathering\venv\Lib\site-packages\gamspy_base\gmsprmNT.txt
    C:\Users\Diego\Documents\GAMS\gamsconfig.yaml
--- GAMS Parameters defined
    Input C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpt13ke5f6\_d0nFgZGBQ4eyFQwyZhTk0A.gms
    Output C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpt13ke5f6\_d0nFgZGBQ4eyFQwyZhTk0A.lst
    ScrDir C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpt13ke5f6\tmpt07pda9n\
    SysDir C:\Users\Diego\Desktop\ExxonMobil\LM_Gathering\venv\Lib\site-packages\gamspy_base\
    LogOption 3
    Trace C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpt13ke5f6\_d0nFgZGBQ4eyFQwyZhTk0A.txt
    License C:\Users\Diego\Documents\GAMSPy\gamspy_license.txt
    OptFile 0
    OptDir C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpt13ke5f6\
    LimRow 

[CONVERTER - INFO] GAMS (.gms) file has been generated under prueba4/iteration_1\Multiphase_network_design.gms


--- Iteration 2 ---
--- _d0nFgZGBQ4eyFQwyZhTk0A.gms(11) 159 Mb
--- Job _d0nFgZGBQ4eyFQwyZhTk0A.gms Start 09/15/25 17:18:10 50.4.1 84b10359 WEX-WEI x86 64bit/MS Windows
--- Applying:
    C:\Users\Diego\Desktop\ExxonMobil\LM_Gathering\venv\Lib\site-packages\gamspy_base\gmsprmNT.txt
    C:\Users\Diego\Documents\GAMS\gamsconfig.yaml
--- GAMS Parameters defined
    MIQCP gurobi
    Input C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpt13ke5f6\_d0nFgZGBQ4eyFQwyZhTk0A.gms
    Output C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpt13ke5f6\_d0nFgZGBQ4eyFQwyZhTk0A.lst
    ScrDir C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpt13ke5f6\tmpt07pda9n\
    SysDir C:\Users\Diego\Desktop\ExxonMobil\LM_Gathering\venv\Lib\site-packages\gamspy_base\
    LogOption 3
    Trace C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpt13ke5f6\_d0nFgZGBQ4eyFQwyZhTk0A.txt
    License C:\Users\Diego\Documents\GAMSPy\gamspy_license.txt
    OptFile 0
    OptDir C:\Users\Public\Documents\Wondershare\C

GamspyException: Return code 3. There was an execution error. Check C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpt13ke5f6\_d0nFgZGBQ4eyFQwyZhTk0A.lst for more information.

=============
Error Summary
=============

**** Exec Error at line 707: Equation infeasible due to rhs value

**** INFEASIBLE EQUATIONS ...

---- one_interval  =E=  Each connection (j, pf, d) must lie between a single 'pw' at each time period 't' 

one_interval(j1,pf1,d1,t1)..  0 =E= 1 ; (LHS = 0, INFES = 1 ****)
     
one_interval(j1,pf1,d1,t2)..  0 =E= 1 ; (LHS = 0, INFES = 1 ****)
     
one_interval(j1,pf1,d1,t13)..  0 =E= 1 ; (LHS = 0, INFES = 1 ****)
     
one_interval(j1,pf1,d1,t15)..  0 =E= 1 ; (LHS = 0, INFES = 1 ****)
     
one_interval(j1,pf1,d1,t16)..  0 =E= 1 ; (LHS = 0, INFES = 1 ****)
     
one_interval(j1,pf1,d1,t18)..  0 =E= 1 ; (LHS = 0, INFES = 1 ****)
     
one_interval(j1,pf1,d1,t19)..  0 =E= 1 ; (LHS = 0, INFES = 1 ****)
     
one_interval(j1,pf1,d1,t21)..  0 =E= 1 ; (LHS = 0, INFES = 1 

In [5]:
vel = ((55335+15810)*0.158987)/((3.1416*((8*0.0254)**2)/4)*24*3600)
print(vel)

4.036955401892067


In [6]:
dp_liq = (1.2562*1.60934)*(0.025/2)*(850*vel*vel)/(8*0.0254)

In [8]:
dp_liq/1000

1.7227399820503617

In [7]:
m.m["loc_x"].records

In [5]:
ylp_inf

10.751957060962651

In [4]:
sel_connections = m.m["sel_pipes"].records

In [5]:
sel_connections

,n,nn,d,element_text,last_selected
0,j2,pf2,d1,,True
1,j3,pf2,d1,,True
2,j5,pf2,d1,,True
3,j7,pf2,d1,,True
4,j8,pf2,d1,,True
5,j10,pf2,d1,,True


In [7]:
last_connections = m.obtain_var_df("x_bar")
last_connections = last_connections[last_connections["level"] > 0.5]

In [4]:
dist_df = m.obtain_var_df("dist", value_col="value")

In [ ]:
dist_df[]

,n,nn,value
0,i1,j1,0.000000
1,i2,j1,0.250000
2,i3,j1,0.500000
3,i4,j1,0.750000
4,i5,j1,2.000000
...,...,...,...
175,j8,pf2,2.490607
176,j9,pf2,1.305038
177,j10,pf2,1.256234
178,j11,pf2,1.256234


In [8]:
last_connections

,n,nn,d,t,level
162,i1,j2,d1,t15,1.0
5925,i2,j2,d1,t18,1.0
11910,i3,j3,d3,t21,1.0
17266,i4,j3,d1,t23,1.0
23706,i5,j5,d3,t1,1.0
29094,i6,j5,d1,t35,1.0
35520,i7,j7,d2,t36,1.0
41429,i8,j8,d2,t37,1.0
47412,i9,j10,d1,t1,1.0
53295,i10,j10,d4,t13,1.0
